La técnica de clustering puede ser un enfoque eficiente para la reducción de la dimansionalidad, en particular como un paso de preprocesamiento antes de un algoritmo de aprendizaje supervisado.

Vamos a utilizar clustering para abordar un conjunto de datos de dígitos del 0 al 9, una versión similiar y simplificada del MNIST con 1797 imágenes de 8x8.

In [1]:
import numpy as np
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=2)

In [3]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

Con estos datos, entrenamos un modelo de regresión logística y lo evaluamos

In [4]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=10_000, random_state=2)
log_reg.fit(X_train_std, y_train)

LogisticRegression(max_iter=10000, random_state=2)

In [5]:
log_reg.score(X_test_std, y_test)

0.9444444444444444

Esta será nuestra línea de base 94.4%. Pruebe si puede mejorar este desempeño usando K-Means como paso de preprocesamiento, reemplazando cada imagen por la distancia a cada uno de los clusters.

Como se tienen 10 dígitos diferentes, existe la tendencia de fijar el número de clusters a 10. Sin embargo, cáda dígito puede ser escrito de diferentes maneras, por tanto es recomendable usar un número mayor de clusters. Probar con un valor arbitrario de k.

In [6]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [7]:
ks = np.arange(6, 11, 1)
for k in ks:
  kmean = KMeans(n_clusters=k, n_init=10, random_state=2).fit(X_train_std)
  predict = kmean.predict(X_test_std)
  transform = kmean.transform(X_train_std)
  score = kmean.score(X_test_std, y_test)
  print(f"Con k={k}:\n\tScore={score}")

Con k=6:
	Score=-21386.80147640933
Con k=7:
	Score=-20211.33219696984
Con k=8:
	Score=-19481.551274783862
Con k=9:
	Score=-19478.82775789401
Con k=10:
	Score=-18075.78033887367


In [35]:
ks2 = np.arange(20, 55, 3)
for k in ks2:
    kmean = KMeans(n_clusters=k, n_init=10, random_state=2).fit(X_train_std)
    transform = kmean.transform(X_train_std)
    transformTest = kmean.transform(X_test_std)
    #print(transform.shape)
    log_reg = LogisticRegression(max_iter=10_000, random_state=2)
    log_reg.fit(transform, y_train)
    score = log_reg.score(transformTest, y_test)
    if score > 0.9444444:
        print(f"Con k={k}:\n\tScore={score}")

Con k=47:
	Score=0.9444444444444444
Con k=50:
	Score=0.9511111111111111


In [9]:
from sklearn.model_selection import GridSearchCV

In [36]:
parametros = {
    "n_clusters" : ks2
}

kmeans = KMeans(random_state=2, n_init=10)
grid = GridSearchCV(estimator=kmeans, param_grid=parametros)
grid.fit(X_train_std, y_train)

GridSearchCV(estimator=KMeans(n_init=10, random_state=2),
             param_grid={'n_clusters': array([20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53])})

Utilice GridSearchCV para encontrar el valor de k 

In [37]:
grid.best_params_

{'n_clusters': 53}